In [403]:
import pandas as pd
import numpy as np
from datetime import date

In [404]:
#Loading the data
today = str(date.today())
df_branch_service = pd.read_json("branch_service_transaction_info.json")
df_customer_transaction = pd.read_json("customer_transaction_info.json")

In [405]:
#Profiling the data
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,NaN
1,TXN-14642,Starmall,HairColor,NaN
2,TXN-60295,SmallMall,FootSpa,NaN
3,TXN-60295,Starmall,FootSpa,NaN
4,TXN-60295,MayMall,FootSpa,NaN
5,TXN-60295,FrankMall,FootSpa,NaN
6,TXN-40462,Starmall,HairColor,NaN
7,TXN-40462,Megamall,HairColor,NaN
8,TXN-08102,RobinsonsMall,HairColor,NaN
9,TXN-08102,SmallMall,HairColor,NaN


In [406]:
df_customer_transaction.head(10)

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,ORTIZ,EDUARDO,1990-07-08
1,TXN-14642,2026-05-26,NIENOW,LEA,2000-11-26
2,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
3,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
4,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
5,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
6,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
7,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
8,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10
9,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10


In [407]:
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(130653, 4)
(130653, 5)


In [408]:
print(df_branch_service['txn_id'].nunique())
print(df_customer_transaction['txn_id'].nunique())

62354
62354


In [409]:
#Dropping duplicates
df_branch_service = df_branch_service.drop_duplicates(subset=['txn_id'])
df_customer_transaction = df_customer_transaction.drop_duplicates(subset=['txn_id'])

In [410]:
#confirming shape
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(62354, 4)
(62354, 5)


In [411]:
#checking null values count per column
df_branch_service.isnull().sum()

txn_id             0
branch_name     8880
service            0
price          11082
dtype: int64

In [412]:
df_customer_transaction.isnull().sum()

txn_id        0
avail_date    0
last_name     0
first_name    0
birthday      0
dtype: int64

In [413]:
#fill up null values with forward fill for branch_name
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'RobinsonsMall', '', None,
       'MayMall', 'FrankMall', 'N/A', 'Megamall'], dtype=object)

In [414]:
#'' and None are empty values, fill them with forward fill
# first, let '' be null
df_branch_service['branch_name'] = df_branch_service.replace('',np.nan).groupby('txn_id')['branch_name'].transform('first')
# then fill null, use forward and backward to fill
df_branch_service['branch_name'] = df_branch_service['branch_name'].ffill().bfill()

In [415]:
#confirming branch_name has no null
df_branch_service.isnull().sum()

txn_id             0
branch_name        0
service            0
price          11082
dtype: int64

In [416]:
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'RobinsonsMall', 'MayMall',
       'FrankMall', 'N/A', 'Megamall'], dtype=object)

In [417]:
# group mean fill prices
df_branch_service['price'] = df_branch_service['price'].fillna(df_branch_service.groupby(['branch_name','service'])['price'].transform('mean'))

In [418]:
#confirming price has no null
df_branch_service.isnull().sum()

txn_id         0
branch_name    0
service        0
price          0
dtype: int64

In [419]:
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,42.651993
1,TXN-14642,Starmall,HairColor,69.583267
2,TXN-60295,SmallMall,FootSpa,80.037329
6,TXN-40462,Starmall,HairColor,69.583267
8,TXN-08102,RobinsonsMall,HairColor,66.735226
12,TXN-64262,SmallMall,Rebond,312.646002
15,TXN-41934,RobinsonsMall,HairColor,66.735226
19,TXN-39050,Starmall,Manicure,43.364281
21,TXN-43150,Starmall,Haircut,51.769018
22,TXN-49423,RobinsonsMall,Rebond,302.828204


In [420]:
df_merged = pd.merge(df_customer_transaction, df_branch_service)

In [421]:
#profiling of merged dataframe
df_merged.isnull().sum()

txn_id         0
avail_date     0
last_name      0
first_name     0
birthday       0
branch_name    0
service        0
price          0
dtype: int64

In [422]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
42144,TXN-18597,2012-05-15,"Jenkins,,,,,,,,",Jewel........,1997-03-28,Megamall,FootSpa,100.121230
896,TXN-45301,2014-03-24,PRICE,NATHANIAL,1991-09-06,RobinsonsMall,Rebond,302.828204
13568,TXN-33858,2017-05-22,ritchie,virginia,2004-01-29,MayMall,HairColor,0.000000
1677,TXN-50137,2014-08-08,WISOKY,TRESSIE,1998-03-15,FrankMall,Pedicure,61.533876
1567,TXN-05373,2023-04-07,ONDRICKA,CHARLEY,1991-11-06,MallOfAsia,FootSpa,78.040113
51039,TXN-06263,2023-03-03,Gerhold,Jaylin,2004-05-30,RobinsonsMall,Pedicure,77.987989
41795,TXN-48121,2026-08-16,"Conroy,,,,,,,,",Cassandra........,1996-05-27,RobinsonsMall,NailColor,30.123790
41588,TXN-28986,2029-07-11,"Dach,,,,,,,,",Madisen........,2006-11-30,Starmall,Haircut,66.123457
55295,TXN-40262,2024-04-14,Nikolaus,Ari,2010-02-17,RobinsonsMall,Rebond,400.231230
16949,TXN-58702,2019-01-28,langosh,aileen,2003-01-18,Megamall,Rebond,0.000000


In [423]:
#filter alphabet only in last_name and first_name
df_merged['last_name'] = df_merged['last_name'].str.replace('\W', '', regex=True)
df_merged['first_name'] = df_merged['first_name'].str.replace('\W', '', regex=True)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\juanc\AppData\Local\Temp\ipykernel_17452\1982014538.py:2: SyntaxWarning: invalid escape sequence '\W'
  df_merged['last_name'] = df_merged['last_name'].str.replace('\W', '', regex=True)
C:\Users\juanc\AppData\Local\Temp\ipykernel_17452\1982014538.py:3: SyntaxWarning: invalid escape sequence '\W'
  df_merged['first_name'] = df_merged['first_name'].str.replace('\W', '', regex=True)


In [424]:
#uppercase for last_name and first_name
df_merged['last_name'] = df_merged['last_name'].str.upper()
df_merged['first_name'] = df_merged['first_name'].str.upper()

In [425]:
#check if no special characters and all uppercase for names
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
16093,TXN-12649,2009-07-13,SCHILLER,JUSTYN,2007-11-14,MallOfAsia,HairColor,0.000000
32054,TXN-35293,2011-11-19,WARD,CHANELLE,2002-02-28,FrankMall,Manicure,55.232400
46328,TXN-23864,2026-05-09,DURGAN,GARDNER,1996-07-23,FrankMall,Pedicure,77.987989
7233,TXN-14691,2029-12-13,BEER,GREYSON,2006-11-03,MallOfAsia,Rebond,316.049754
59352,TXN-55688,2019-03-15,HERMAN,ANSLEY,1995-05-31,MayMall,NailColor,30.123790
21543,TXN-61186,2017-01-12,HUELS,SHERIDAN,2003-01-23,Starmall,Pedicure,0.000000
58652,TXN-11107,2007-06-30,EICHMANN,SANDRINE,1994-09-07,FrankMall,NailColor,30.123790
27057,TXN-50729,2014-06-15,FAY,REECE,1996-12-06,Megamall,Rebond,400.231230
53643,TXN-63005,2028-09-17,RIPPIN,CLEORA,1995-03-04,MayMall,Manicure,55.232400
27501,TXN-42381,2021-10-20,MCLAUGHLIN,ERNESTO,2009-03-01,RobinsonsMall,Rebond,400.231230


In [426]:
#profiling date columns
print(df_merged['birthday'].min())
print(df_merged['birthday'].max())

print(df_merged['avail_date'].min())
print(df_merged['avail_date'].max())

print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

print(df_merged[(df_merged['avail_date'] <= df_merged['birthday'])])

1990-01-01
2010-12-30
2005-01-01
2030-12-30
<bound method NDFrame.describe of 0        2030-09-08
1        2026-05-26
2        2006-09-25
3        2021-08-21
4        2010-04-03
            ...    
62349    2022-11-21
62350    2020-04-14
62351    2030-01-26
62352    2021-02-27
62353    2025-01-05
Name: avail_date, Length: 62354, dtype: object>
<bound method NDFrame.describe of 0        1990-07-08
1        2000-11-26
2        1993-05-22
3        2002-11-25
4        2003-07-10
            ...    
62349    1997-03-06
62350    1996-05-28
62351    1996-05-28
62352    1995-03-29
62353    1993-01-10
Name: birthday, Length: 62354, dtype: object>
          txn_id  avail_date  last_name first_name    birthday    branch_name  \
9      TXN-49423  2008-08-13      BERGE    RASHEED  2010-02-05  RobinsonsMall   
17     TXN-35244  2005-11-23     HAMMES     EUNICE  2006-06-13      SmallMall   
97     TXN-23898  2005-05-28   BOTSFORD     GIANNI  2009-03-30  RobinsonsMall   
101    TXN-17974  2008-04-12  

In [427]:
#data type of birthday and avail_date should be datetime instead of object
df_merged['avail_date'] = pd.to_datetime(df_merged['avail_date'], format='%Y-%m-%d')
df_merged['birthday'] = pd.to_datetime(df_merged['birthday'], format='%Y-%m-%d')

#confirming
print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

<bound method NDFrame.describe of 0       2030-09-08
1       2026-05-26
2       2006-09-25
3       2021-08-21
4       2010-04-03
           ...    
62349   2022-11-21
62350   2020-04-14
62351   2030-01-26
62352   2021-02-27
62353   2025-01-05
Name: avail_date, Length: 62354, dtype: datetime64[ns]>
<bound method NDFrame.describe of 0       1990-07-08
1       2000-11-26
2       1993-05-22
3       2002-11-25
4       2003-07-10
           ...    
62349   1997-03-06
62350   1996-05-28
62351   1996-05-28
62352   1995-03-29
62353   1993-01-10
Name: birthday, Length: 62354, dtype: datetime64[ns]>


In [428]:
#removing rows with later date than current date
df_merged = df_merged[(df_merged['avail_date'] <= today) & (df_merged['birthday'] <= today)]

In [429]:
#removing rows when avail_date happens before birthday
df_merged = df_merged[(df_merged['avail_date'] > df_merged['birthday'])]

In [430]:
#confirming
print(df_merged['birthday'].max())
print(df_merged['avail_date'].max())

print(df_merged[(df_merged['avail_date'] <= df_merged['birthday'])])

2010-12-30 00:00:00
2023-11-23 00:00:00
Empty DataFrame
Columns: [txn_id, avail_date, last_name, first_name, birthday, branch_name, service, price]
Index: []


In [431]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
11427,TXN-48083,2017-09-18,COLLIER,RODGER,1995-08-09,Starmall,Rebond,0.000000
34538,TXN-40954,2017-07-25,BOYLE,HASSAN,1995-09-28,Megamall,Pedicure,77.987989
61158,TXN-11428,2014-01-09,DICKINSON,MANUELA,1990-06-22,Starmall,FootSpa,100.121230
38928,TXN-31922,2019-07-20,FLATLEY,LINCOLN,2009-06-11,MayMall,Pedicure,77.987989
52526,TXN-55728,2007-08-27,MAGGIO,SANTA,2001-07-08,Starmall,FootSpa,100.121230
28935,TXN-44745,2015-12-05,MULLER,RACHAEL,1995-11-08,MayMall,HairColor,88.093930
41698,TXN-55975,2018-08-27,LIND,KRISTINA,2000-09-26,FrankMall,NailColor,30.123790
50421,TXN-32639,2015-10-09,KSHLERIN,ROSEMARIE,2008-11-15,RobinsonsMall,NailColor,30.123790
24987,TXN-58801,2013-07-15,LEFFLER,SEAN,1998-07-10,MallOfAsia,FootSpa,100.121230
60456,TXN-48730,2015-05-31,HANE,SAIGE,1993-06-04,Starmall,HairColor,88.093930


In [432]:
df_merged.shape

(43136, 8)

In [433]:
import psycopg2

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [435]:
con = psycopg2.connect("user=postgres password='postgres'")

con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [436]:
cursor          = con.cursor()

name_Database   = "transaction"

In [437]:
sqlCreateDatabase = "create database "+name_Database+";"

In [438]:
cursor.execute(sqlCreateDatabase)

In [440]:
cursor.execute("create table if not exists transaction_table (txn_id varchar(45) not null primary key,avail_data date, last_name varchar(20),birthday date,branch_name varchar(30), service varchar(30), price double precision)")